In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from multiprocessing import Pool

### Function Used

In [2]:
def model_info(x, y, test_date, data_tested):
    cm = confusion_matrix(x,y)
    acc = accuracy_score(x,y)
    rec = recall_score(x,y)
    rec_0 = recall_score(x,y, pos_label=0)
    data = [{"Accuracy":acc,"Specificity":rec_0, "Sensitivity":rec}]
    df = pd.DataFrame(data)
    df['test_date'] = test_date
    df['data_tested'] = data_tested
    if len(x.value_counts()) == 1:
        l30 = x.value_counts()[0]
        df['greter_than_30'] = '0'
        df['less_than_30'] = l30
    else:
        g30 = x.value_counts()[1]
        l30 = x.value_counts()[0]
        df['greter_than_30'] = g30
        df['less_than_30'] = l30
    return (df)

### Load Data

In [3]:
df = pd.read_csv("15min_dateset.csv")
df.drop('Unnamed: 0', inplace = True, axis = 1)
print(len(df['EXP'].unique()))
df.head(2)

32


,Date,Type,from,EXP,jul_date,hour_angle,max_wind_speed,avg_wind_speed,R1,R2,...,HYBL_one,HYBL_two,HYBL_three,HYBL_four,HYBL_five,diff_temp,geo_cbl,veg_sfc,best_4_layer,ws_700
0,2017-01-07 06:00:00,ANDE,MESONET,1/7/17,7,-1.570795,2.332608,1.295893,2.630564,2.322274,...,0.629230,0.903721,3.610055,5.011670,6.054984,0.739,-5000.0,40.293,23.757,16.787335
1,2017-01-07 06:15:00,ANDE,MESONET,1/7/17,7,-1.505345,1.555072,0.647947,2.055383,1.813005,...,0.748575,1.493288,3.643217,5.232982,6.394255,1.050,-5000.0,40.293,23.959,16.195689


### Create Column for classification

In [4]:
df["max_th"] = np.where(df["max_wind_speed"] > 30, 1, 0 )
table = df["max_th"].value_counts()
total = table[1] + table[0]
print(table)
print('% greater than 30:', round(table[1]/total,3))
print('% less than 30:', round(table[0]/total,3))

0    69019
1     3630
Name: max_th, dtype: int64
% greater than 30: 0.05
% less than 30: 0.95


In [5]:
df2 = df[['Type', 'from', 'EXP',
       'max_wind_speed', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700', 'max_th']]
le = LabelEncoder()
df2['from'] = le.fit_transform(df2['from'])
df2.head(2)

,Type,from,EXP,max_wind_speed,R1,R2,Pressure_reduced_to_MSL_.Pa..0.MSL,Derived_radar_reflectivity_.dB..1.HYBL,u.component_of_wind_.m.s..85000.ISBL,v.component_of_wind_.m.s..85000.ISBL,...,HYBL_two,HYBL_three,HYBL_four,HYBL_five,diff_temp,geo_cbl,veg_sfc,best_4_layer,ws_700,max_th
0,ANDE,1,1/7/17,2.332608,2.630564,2.322274,102879.7,0.0,5.447,-1.457,...,0.903721,3.610055,5.011670,6.054984,0.739,-5000.0,40.293,23.757,16.787335,0
1,ANDE,1,1/7/17,1.555072,2.055383,1.813005,102951.0,0.0,5.629,-1.464,...,1.493288,3.643217,5.232982,6.394255,1.050,-5000.0,40.293,23.959,16.195689,0


### Grid Search for best parameters

In [11]:
X = df2.drop(['max_wind_speed', 'Type', 'EXP','max_th'], 1)
Y = df2.max_th.values

pram = {'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate': [.01,0.1,0.3],
        'n_estimators': [100,250,500]}

In [13]:
clf = GridSearchCV(XGBClassifier(objective='binary:logistic' , random_state=19), pram, cv=3, n_jobs=-1, verbose=3, scoring='roc_auc')
clf.fit(X,Y)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 20.4min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed: 40.3min
[Parallel(n_jobs=-1)]: Done 729 out of 729 | elapsed: 64.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=19, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 4, 5

In [14]:
clf.best_score_, clf.best_params_,clf.scorer_

(0.9323359560570017,
 {'colsample_bytree': 0.6,
  'learning_rate': 0.01,
  'max_depth': 5,
  'n_estimators': 500,
  'subsample': 0.6},
 make_scorer(roc_auc_score, needs_threshold=True))

### XGB Model Non-resampling

In [16]:
def rf_model(x):
    test_1 = df2[df2.EXP == x]
    train_1 = df2[df2.EXP != x]
    train_1 = train_1[['Type', 'from', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700','max_th','max_wind_speed']]


    test_1 = test_1[['Type', 'from', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700','max_th','max_wind_speed']]
    
    
    x_train = train_1.drop('max_th', 1)
    x_train.drop(['Type','max_wind_speed'], axis=1, inplace=True)
    y_train = train_1.max_th
    x_test = test_1.drop('max_th', 1)
    x_test_copy = x_test.copy()
    x_test.drop(['Type','max_wind_speed'], axis=1, inplace=True)
    y_test = test_1.max_th
    
    model = XGBClassifier(objective='binary:logistic', max_depth=5, n_estimators=500,colsample_bytree= 0.6 ,
                          learning_rate =  0.01, subsample = 0.6, random_state=19)
    model.fit(x_train, y_train)
    
    pred_train = model.predict(x_train)
    train_info = model_info(y_train, pred_train, x , "train")

    pred_test = model.predict(x_test)
    test_info = model_info(y_test, pred_test, x , "test")

    table = train_info.append(test_info)
    
    print(x)
    print(table)
    print("--" * 40)
    prod = model.predict_proba(x_test)
    prod_data = pd.DataFrame(prod, columns=["less_30","greater_30"])
    df4 = pd.DataFrame(y_test)
    df4 = df4.rename(columns={'max_th': 'actual'})
    df4 = df4.reset_index()
    df4.drop('index', axis=1, inplace=True)
    x_test_copy = x_test_copy.reset_index()
    x_test_copy.drop("index", axis=1, inplace=True)
    df5 = pd.concat([x_test_copy, df4, prod_data], axis=1)
    pred_df = pd.DataFrame(pred_test, columns=["pred"])
    df6 = pd.concat([df5, pred_df], axis=1)
    df6['From'] = np.where(df6['from'] == 1, 'Mesonet', 'ASOS')
    df6.drop('from', inplace=True, axis=1)
    df6['test_date'] = x
    return(table, df6)



In [17]:
%%time
table = pd.DataFrame()
data = pd.DataFrame()
for i in df['EXP'].unique():
    t,d = rf_model(i)
    table = table.append(t)
    data = data.append(d)

1/7/17
   Accuracy  Specificity  Sensitivity test_date data_tested greter_than_30  \
0  0.965582     0.995929     0.404959    1/7/17       train           3630   
0  1.000000     1.000000     0.000000    1/7/17        test              0   

   less_than_30  
0         67060  
0          1959  
--------------------------------------------------------------------------------
2/9/17
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.965943     0.995911     0.404529    2/9/17       train            3577   
0  0.973314     0.991036     0.301887    2/9/17        test              53   

   less_than_30  
0         67011  
0          2008  
--------------------------------------------------------------------------------
2/13/17
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.970364     0.997101     0.385238   2/13/17       train            3089   
0  0.759325     0.993644     0.146026   2/13/17        test             541  

9/4/18
   Accuracy  Specificity  Sensitivity test_date data_tested greter_than_30  \
0  0.965384     0.996307     0.395592    9/4/18       train           3630   
0  1.000000     1.000000     0.000000    9/4/18        test              0   

   less_than_30  
0         66886  
0          2133  
--------------------------------------------------------------------------------
9/6/18
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.965490     0.995898     0.405077    9/6/18       train            3624   
0  0.997315     1.000000     0.000000    9/6/18        test               6   

   less_than_30  
0         66790  
0          2229  
--------------------------------------------------------------------------------
9/10/18
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.965107     0.995911     0.397793   9/10/18       train            3625   
0  0.997790     1.000000     0.000000   9/10/18        test               5  

In [18]:
data.tail()

,Type,R1,R2,Pressure_reduced_to_MSL_.Pa..0.MSL,Derived_radar_reflectivity_.dB..1.HYBL,u.component_of_wind_.m.s..85000.ISBL,v.component_of_wind_.m.s..85000.ISBL,wind_speed_85000,wind_shear_85000,Wind_speed_.gust._.m.s..0.SFC,...,veg_sfc,best_4_layer,ws_700,max_wind_speed,actual,less_30,greater_30,pred,From,test_date
2331,WANT,0.098920,0.157647,100027.320,0.0,32.948,-5.489,33.402093,23.859866,37.742,...,40.175,21.381,17.640080,37.904880,1,0.775173,0.224827,0,Mesonet,2/24/19
2332,WANT,0.098133,0.156265,100043.773,0.0,33.102,-5.860,33.616692,24.105445,38.260,...,40.175,21.517,19.861816,46.457776,1,0.777900,0.222100,0,Mesonet,2/24/19
2333,WANT,0.098374,0.154478,100029.609,0.0,33.439,-5.925,33.959864,24.255558,39.137,...,40.175,21.800,22.217660,37.710496,1,0.778572,0.221428,0,Mesonet,2/24/19
2334,WANT,0.097222,0.153720,100029.547,0.0,33.765,-5.858,34.269394,24.277645,40.062,...,40.175,21.817,22.936901,39.071184,1,0.775179,0.224821,0,Mesonet,2/24/19
2335,WANT,0.094813,0.153293,100049.734,0.0,33.951,-5.796,34.442184,24.231285,40.566,...,40.175,21.817,23.463382,36.349808,1,0.772047,0.227953,0,Mesonet,2/24/19


In [19]:
table

,Accuracy,Specificity,Sensitivity,test_date,data_tested,greter_than_30,less_than_30
0,0.965582,0.995929,0.404959,1/7/17,train,3630,67060
0,1.000000,1.000000,0.000000,1/7/17,test,0,1959
0,0.965943,0.995911,0.404529,2/9/17,train,3577,67011
0,0.973314,0.991036,0.301887,2/9/17,test,53,2008
0,0.970364,0.997101,0.385238,2/13/17,train,3089,67603
...,...,...,...,...,...,...,...
0,0.913189,0.990556,0.089385,12/17/18,test,179,1906
0,0.968097,0.996221,0.423155,12/22/18,train,3455,66946
0,0.926157,0.993247,0.131429,12/22/18,test,175,2073
0,0.967446,0.996132,0.395113,2/24/19,train,3356,66957


In [20]:
#table.to_csv('XGB_CLASS_greater_30_no_sample_stats.csv')

In [21]:
#data.to_csv('XGB_CLASS_greater_30_no_sample_data.csv')

### XGB Model resampling

In [22]:
def rf_model2(x):
    test_1 = df2[df2.EXP == x]
    train_1 = df2[df2.EXP != x]
    train_1 = train_1[['Type', 'from', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700','max_th','max_wind_speed']]


    test_1 = test_1[['Type', 'from', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700','max_th','max_wind_speed']]
    
    df_0 = train_1[train_1['max_th'] == 0]
    df_1 = train_1[train_1['max_th'] == 1]
    df_under = df_0.sample(9000 ,random_state=19)
    df_new = pd.concat([df_under, df_1], axis = 0)
    
    
    x_train = df_new.drop('max_th', 1)
    x_train.drop(['Type','max_wind_speed'], axis=1, inplace=True)
    y_train = df_new.max_th
    x_test = test_1.drop('max_th', 1)
    x_test_copy = x_test.copy()
    x_test.drop(['Type','max_wind_speed'], axis=1, inplace=True)
    y_test = test_1.max_th
    
    model = XGBClassifier(objective='binary:logistic', max_depth=5, n_estimators=500,colsample_bytree= 0.6 ,
                          learning_rate =  0.01, subsample = 0.6, random_state=19)
    model.fit(x_train, y_train)
    
    pred_train = model.predict(x_train)
    train_info = model_info(y_train, pred_train, x , "train")

    pred_test = model.predict(x_test)
    test_info = model_info(y_test, pred_test, x , "test")

    table = train_info.append(test_info)
    
    print(x)
    print(table)
    print("--" * 40)
    prod = model.predict_proba(x_test)
    prod_data = pd.DataFrame(prod, columns=["less_30","greater_30"])
    df4 = pd.DataFrame(y_test)
    df4 = df4.rename(columns={'max_th': 'actual'})
    df4 = df4.reset_index()
    df4.drop('index', axis=1, inplace=True)
    x_test_copy = x_test_copy.reset_index()
    x_test_copy.drop("index", axis=1, inplace=True)
    df5 = pd.concat([x_test_copy, df4, prod_data], axis=1)
    pred_df = pd.DataFrame(pred_test, columns=["pred"])
    df6 = pd.concat([df5, pred_df], axis=1)
    df6['From'] = np.where(df6['from'] == 1, 'Mesonet', 'ASOS')
    df6.drop('from', inplace=True, axis=1)
    df6['test_date'] = x
    return(table, df6)

In [23]:
%%time
table2 = pd.DataFrame()
data2 = pd.DataFrame()
for i in df['EXP'].unique():
    t,d = rf_model2(i)
    table2 = table2.append(t)
    data2 = data2.append(d)

1/7/17
   Accuracy  Specificity  Sensitivity test_date data_tested greter_than_30  \
0  0.919002     0.933556      0.88292    1/7/17       train           3630   
0  0.995406     0.995406      0.00000    1/7/17        test              0   

   less_than_30  
0          9000  
0          1959  
--------------------------------------------------------------------------------
2/9/17
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.923034     0.937667     0.886218    2/9/17       train            3577   
0  0.939350     0.943725     0.773585    2/9/17        test              53   

   less_than_30  
0          9000  
0          2008  
--------------------------------------------------------------------------------
2/13/17
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.929275     0.948667     0.872774   2/13/17       train            3089   
0  0.785386     0.808616     0.724584   2/13/17        test             541  

9/4/18
   Accuracy  Specificity  Sensitivity test_date data_tested greter_than_30  \
0  0.924545     0.939444     0.887603    9/4/18       train           3630   
0  1.000000     1.000000     0.000000    9/4/18        test              0   

   less_than_30  
0          9000  
0          2133  
--------------------------------------------------------------------------------
9/6/18
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.924430        0.938     0.890728    9/6/18       train            3624   
0  0.997315        1.000     0.000000    9/6/18        test               6   

   less_than_30  
0          9000  
0          2229  
--------------------------------------------------------------------------------
9/10/18
   Accuracy  Specificity  Sensitivity test_date data_tested  greter_than_30  \
0  0.923485     0.939222     0.884414   9/10/18       train            3625   
0  0.997790     1.000000     0.000000   9/10/18        test               5  

In [24]:
table2

,Accuracy,Specificity,Sensitivity,test_date,data_tested,greter_than_30,less_than_30
0,0.919002,0.933556,0.882920,1/7/17,train,3630,9000
0,0.995406,0.995406,0.000000,1/7/17,test,0,1959
0,0.923034,0.937667,0.886218,2/9/17,train,3577,9000
0,0.939350,0.943725,0.773585,2/9/17,test,53,2008
0,0.929275,0.948667,0.872774,2/13/17,train,3089,9000
...,...,...,...,...,...,...,...
0,0.786091,0.806925,0.564246,12/17/18,test,179,1906
0,0.927660,0.943000,0.887699,12/22/18,train,3455,9000
0,0.832740,0.849493,0.634286,12/22/18,test,175,2073
0,0.926433,0.944222,0.878725,2/24/19,train,3356,9000


In [25]:
data.tail()

,Type,R1,R2,Pressure_reduced_to_MSL_.Pa..0.MSL,Derived_radar_reflectivity_.dB..1.HYBL,u.component_of_wind_.m.s..85000.ISBL,v.component_of_wind_.m.s..85000.ISBL,wind_speed_85000,wind_shear_85000,Wind_speed_.gust._.m.s..0.SFC,...,veg_sfc,best_4_layer,ws_700,max_wind_speed,actual,less_30,greater_30,pred,From,test_date
2331,WANT,0.098920,0.157647,100027.320,0.0,32.948,-5.489,33.402093,23.859866,37.742,...,40.175,21.381,17.640080,37.904880,1,0.775173,0.224827,0,Mesonet,2/24/19
2332,WANT,0.098133,0.156265,100043.773,0.0,33.102,-5.860,33.616692,24.105445,38.260,...,40.175,21.517,19.861816,46.457776,1,0.777900,0.222100,0,Mesonet,2/24/19
2333,WANT,0.098374,0.154478,100029.609,0.0,33.439,-5.925,33.959864,24.255558,39.137,...,40.175,21.800,22.217660,37.710496,1,0.778572,0.221428,0,Mesonet,2/24/19
2334,WANT,0.097222,0.153720,100029.547,0.0,33.765,-5.858,34.269394,24.277645,40.062,...,40.175,21.817,22.936901,39.071184,1,0.775179,0.224821,0,Mesonet,2/24/19
2335,WANT,0.094813,0.153293,100049.734,0.0,33.951,-5.796,34.442184,24.231285,40.566,...,40.175,21.817,23.463382,36.349808,1,0.772047,0.227953,0,Mesonet,2/24/19


In [26]:
#table2.to_csv('XGB_CLASS_greater_30_sample9000_stats.csv')

In [27]:
#data2.to_csv('XGB_CLASS_greater_30_sample9000_data.csv')